Load & de-duplicate on Uniq Id.

Clean key text fields (normalize spaces, strip HTML-ish tags).

Parse categories into a list + a main category.

Normalize prices and create a price column from Selling Price / List Price.

Filter to usable rows (must have Product Name + Image; optional price/category).

Create a single product_text field that combines all useful product attribute

In [4]:
import pandas as pd
import numpy as np

raw_path = "Amazon_data.csv"

df = pd.read_csv(raw_path)
print(df.shape)
df.head()

(10002, 28)


,Uniq Id,Product Name,Brand Name,Asin,Category,Upc Ean Code,List Price,Selling Price,Quantity,Model Number,...,Product Url,Stock,Product Details,Dimensions,Color,Ingredients,Direction To Use,Is Amazon Seller,Size Quantity Variant,Product Description
0,4c69b61db1fc16e7013b43fc926e502d,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",NaN,NaN,Sports & Outdoors | Outdoor Recreation | Skate...,NaN,NaN,$237.68,NaN,NaN,...,https://www.amazon.com/DB-Longboards-CoreFlex-...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
1,66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, ...",NaN,NaN,Toys & Games | Learning & Education | Science ...,NaN,NaN,$99.95,NaN,55324,...,https://www.amazon.com/Electronic-Circuits-Cla...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
2,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,NaN,NaN,Toys & Games | Arts & Crafts | Craft Kits,NaN,NaN,$34.99,NaN,NaN,...,https://www.amazon.com/3Doodler-Plastic-Innova...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
3,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,NaN,NaN,Toys & Games | Hobbies | Models & Model Kits |...,NaN,NaN,$28.91,NaN,142,...,https://www.amazon.com/Guillow-Airplane-Design...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
4,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock- Collage 500 pc Puzzle,NaN,NaN,Toys & Games | Puzzles | Jigsaw Puzzles,NaN,NaN,$17.49,NaN,62151,...,https://www.amazon.com/Woodstock-Collage-500-p...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN


In [6]:
df.columns.tolist()

['Uniq Id',
 'Product Name',
 'Brand Name',
 'Asin',
 'Category',
 'Upc Ean Code',
 'List Price',
 'Selling Price',
 'Quantity',
 'Model Number',
 'About Product',
 'Product Specification',
 'Technical Details',
 'Shipping Weight',
 'Product Dimensions',
 'Image',
 'Variants',
 'Sku',
 'Product Url',
 'Stock',
 'Product Details',
 'Dimensions',
 'Color',
 'Ingredients',
 'Direction To Use',
 'Is Amazon Seller',
 'Size Quantity Variant',
 'Product Description']

In [8]:
df.isna().mean().sort_values(ascending=False)

Product Description      1.000000
Sku                      1.000000
Brand Name               1.000000
Asin                     1.000000
Size Quantity Variant    1.000000
List Price               1.000000
Direction To Use         1.000000
Quantity                 1.000000
Ingredients              1.000000
Color                    1.000000
Dimensions               1.000000
Product Details          1.000000
Stock                    1.000000
Upc Ean Code             0.996601
Product Dimensions       0.952110
Variants                 0.752250
Model Number             0.177165
Product Specification    0.163167
Shipping Weight          0.113777
Category                 0.082983
Technical Details        0.078984
About Product            0.027295
Selling Price            0.010698
Image                    0.000000
Product Name             0.000000
Product Url              0.000000
Is Amazon Seller         0.000000
Uniq Id                  0.000000
dtype: float64

In [10]:
# Drop exact duplicate rows
df = df.drop_duplicates()

# Drop duplicate product IDs if any (keep first)
if "Uniq Id" in df.columns:
    df = df.drop_duplicates(subset=["Uniq Id"], keep="first")

# Keep only rows with at least a name and an image
df = df[df["Product Name"].notna()]

if "Image" in df.columns:
    df = df[df["Image"].notna()]

df = df.reset_index(drop=True)
print(df.shape)


(10002, 28)


In [12]:
def clean_price(x):
    if pd.isna(x):
        return np.nan
    x = str(x)
    x = x.replace("$", "").replace(",", "").strip()
    return pd.to_numeric(x, errors="coerce")

for col in ["List Price", "Selling Price"]:
    if col in df.columns:
        df[col + " (num)"] = df[col].apply(clean_price)

df[["List Price", "List Price (num)", "Selling Price", "Selling Price (num)"]].head()


,List Price,List Price (num),Selling Price,Selling Price (num)
0,NaN,NaN,$237.68,237.68
1,NaN,NaN,$99.95,99.95
2,NaN,NaN,$34.99,34.99
3,NaN,NaN,$28.91,28.91
4,NaN,NaN,$17.49,17.49


<span style="color: #800080; font-weight: bold;"># Extract main category</span>


In [14]:
def get_main_category(cat):
    if pd.isna(cat):
        return np.nan
    return str(cat).split("|")[0].strip()

if "Category" in df.columns:
    df["Main Category"] = df["Category"].apply(get_main_category)

df[["Category", "Main Category"]].head(10)


,Category,Main Category
0,Sports & Outdoors | Outdoor Recreation | Skate...,Sports & Outdoors
1,Toys & Games | Learning & Education | Science ...,Toys & Games
2,Toys & Games | Arts & Crafts | Craft Kits,Toys & Games
3,Toys & Games | Hobbies | Models & Model Kits |...,Toys & Games
4,Toys & Games | Puzzles | Jigsaw Puzzles,Toys & Games
5,NaN,NaN
6,"Clothing, Shoes & Jewelry | Costumes & Accesso...","Clothing, Shoes & Jewelry"
7,Toys & Games | Arts & Crafts | Drawing & Paint...,Toys & Games
8,Home & Kitchen | Home Décor | Window Treatment...,Home & Kitchen
9,NaN,NaN


In [16]:
TEXT_COLS = [
    "Product Name",
    "Brand Name",
    "Category",
    "Main Category",
    "About Product",
    "Product Specification",
    "Technical Details",
    "Product Details",
    "Product Description",
    "Color",
    "Ingredients",
    "Direction To Use",
    "Size Quantity Variant",
]

for col in TEXT_COLS:
    if col in df.columns:
        df[col] = df[col].fillna("").astype(str).str.strip()


<span style="color: #800080; font-weight: bold;"># Build a unified product_text field</span>


In [18]:
def build_product_text(row):
    parts = []

    # --- Basic identity ---
    name = row.get("Product Name", "")
    brand = row.get("Brand Name", "")
    main_cat = row.get("Main Category", "")
    full_cat = row.get("Category", "")
    price = row.get("Selling Price", "")
    color = row.get("Color", "")
    size_variant = row.get("Size Quantity Variant", "")

    if name:
        parts.append(f"Product Name: {name}")
    if brand:
        parts.append(f"Brand: {brand}")
    if main_cat:
        parts.append(f"Main Category: {main_cat}")
    elif full_cat:
        parts.append(f"Category: {full_cat}")
    if color:
        parts.append(f"Color: {color}")
    if size_variant:
        parts.append(f"Size / Quantity: {size_variant}")
    if price:
        parts.append(f"Selling Price: {price}")

    # --- Descriptive fields ---
    about = row.get("About Product", "")
    spec = row.get("Product Specification", "")
    tech = row.get("Technical Details", "")
    details = row.get("Product Details", "")
    desc = row.get("Product Description", "")
    ingredients = row.get("Ingredients", "")
    directions = row.get("Direction To Use", "")

    if about:
        parts.append(f"About Product: {about}")
    if spec:
        parts.append(f"Product Specification: {spec}")
    if tech:
        parts.append(f"Technical Details: {tech}")
    if details:
        parts.append(f"Product Details: {details}")
    if desc:
        parts.append(f"Product Description: {desc}")
    if ingredients:
        parts.append(f"Ingredients: {ingredients}")
    if directions:
        parts.append(f"Directions To Use: {directions}")

    # Join with double newlines for readability
    return "\n\n".join(parts)

df["product_text"] = df.apply(build_product_text, axis=1)

df[["Product Name", "Brand Name", "Main Category", "product_text"]].head(3)


,Product Name,Brand Name,Main Category,product_text
0,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",,Sports & Outdoors,Product Name: DB Longboards CoreFlex Crossbow ...
1,"Electronic Snap Circuits Mini Kits Classpack, ...",,Toys & Games,Product Name: Electronic Snap Circuits Mini Ki...
2,3Doodler Create Flexy 3D Printing Filament Ref...,,Toys & Games,Product Name: 3Doodler Create Flexy 3D Printin...


<span style="color: #800080; font-weight: bold;"># Filter to “usable” products for RAG</span>

Let’s keep items where product_text is not too short and Image exists.

In [20]:
# Length of combined text
df["text_len"] = df["product_text"].str.len()

usable = df[
    (df["text_len"] > 50) & 
    df["Image"].notna() &
    (df["Image"].astype(str).str.len() > 0)
].copy()

usable = usable.reset_index(drop=True)
print(usable.shape)

usable[["Product Name", "Image", "text_len"]].head(5)


(10002, 33)


,Product Name,Image,text_len
0,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",https://images-na.ssl-images-amazon.com/images...,1462
1,"Electronic Snap Circuits Mini Kits Classpack, ...",https://images-na.ssl-images-amazon.com/images...,2486
2,3Doodler Create Flexy 3D Printing Filament Ref...,https://images-na.ssl-images-amazon.com/images...,2234
3,Guillow Airplane Design Studio with Travel Cas...,https://images-na.ssl-images-amazon.com/images...,1268
4,Woodstock- Collage 500 pc Puzzle,https://images-na.ssl-images-amazon.com/images...,686


In [22]:
processed_path_csv = "amazon_processed.csv"
processed_path_parquet = "amazon_processed.parquet"

usable.to_csv(processed_path_csv, index=False)
usable.to_parquet(processed_path_parquet, index=False)

usable.head()

,Uniq Id,Product Name,Brand Name,Asin,Category,Upc Ean Code,List Price,Selling Price,Quantity,Model Number,...,Ingredients,Direction To Use,Is Amazon Seller,Size Quantity Variant,Product Description,List Price (num),Selling Price (num),Main Category,product_text,text_len
0,4c69b61db1fc16e7013b43fc926e502d,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",,NaN,Sports & Outdoors | Outdoor Recreation | Skate...,NaN,NaN,$237.68,NaN,NaN,...,,,Y,,,NaN,237.68,Sports & Outdoors,Product Name: DB Longboards CoreFlex Crossbow ...,1462
1,66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, ...",,NaN,Toys & Games | Learning & Education | Science ...,NaN,NaN,$99.95,NaN,55324,...,,,Y,,,NaN,99.95,Toys & Games,Product Name: Electronic Snap Circuits Mini Ki...,2486
2,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,,NaN,Toys & Games | Arts & Crafts | Craft Kits,NaN,NaN,$34.99,NaN,NaN,...,,,Y,,,NaN,34.99,Toys & Games,Product Name: 3Doodler Create Flexy 3D Printin...,2234
3,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,,NaN,Toys & Games | Hobbies | Models & Model Kits |...,NaN,NaN,$28.91,NaN,142,...,,,Y,,,NaN,28.91,Toys & Games,Product Name: Guillow Airplane Design Studio w...,1268
4,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock- Collage 500 pc Puzzle,,NaN,Toys & Games | Puzzles | Jigsaw Puzzles,NaN,NaN,$17.49,NaN,62151,...,,,Y,,,NaN,17.49,Toys & Games,Product Name: Woodstock- Collage 500 pc Puzzle...,686
